In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')


In [2]:
login = pd.read_csv('key.csv')
consumer_key = login['key'][0]
consumer_secret = login['secret'][0]
access_token = login['token'][0]
access_token_secret = login['token_secret'][0]

In [3]:
#auth obj
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret)

#set access token
authenticate.set_access_token(access_token, access_token_secret)

#create api obj
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [25]:
#Extract 100 tweets from twitter user
def getTweetFromUser(query,n,lang):
    return api.user_timeline(screen_name=query, count=n, lang=lang, tweet_mode="extended")

def getTweet(query,n):
    return api.search(q=query, count=n, tweet_mode="extended")

posts = getTweet("#ม็อบ18พฤศจิกา",50)


for tweet in posts[0:5]:
    print(tweet.full_text+'\n')

RT @fuckingpla: today #ม็อบ18พฤศจิกา https://t.co/SwyU9Ajs2h

RT @plagimlim: โอ้ย น่ารักกกกก ปังมากแม่ นักข่าวมาเอาคลิปแบบนี้ลงบ้างดิ คนที่ไม่ได้เล่นทวิตเขาจะได้เห็นว่าม็อบไม่ได้รุนแรงอย่างที่เขาคิดกัน…

RT @FreeYOUTHth: “ราษฎรโต้กลับ!”
—
#18พฤศจิกาไปราษฎร์ประสงค์
#ม็อบ18พฤศจิกา https://t.co/WOFCDkPOfy

RT @BunlengWan: มันหมดยุคที่พวกมึงจะข่มขู่ไม่ให้คนไปม็อบด้วยการใช้ความรุนแรงแล้ว…ยิ่งมึงทำคนของพวกกูมากเท่าไร สิ่งที่เกิดขึ้นในจิตใจของประช…

RT @GGritchy: Quote of the day #ม็อบ18พฤศจิกา https://t.co/7RZrlu7OFO



In [26]:
#create dataframe
df = pd.DataFrame([tweet.full_text for tweet in posts], columns= ['Tweets'])

df.head()

,Tweets
0,RT @fuckingpla: today #ม็อบ18พฤศจิกา https://t...
1,RT @plagimlim: โอ้ย น่ารักกกกก ปังมากแม่ นักข่...
2,RT @FreeYOUTHth: “ราษฎรโต้กลับ!”\n—\n#18พฤศจิก...
3,RT @BunlengWan: มันหมดยุคที่พวกมึงจะข่มขู่ไม่ใ...
4,RT @GGritchy: Quote of the day #ม็อบ18พฤศจิกา ...


In [27]:
#clean the text

#create function to clean the tweets
def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9_]+:', '', text) # remove #mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # remove #mentions
    text = re.sub(r'#','',text) # remove hashtag
    text = re.sub(r'RT[\s]+','', text) # remove RT
    text = re.sub(r'https?:\/\/\S+','', text) # remove hyper link
    return text

#clean the text
df['Tweets'] = df['Tweets'].apply(cleanText)

df

,Tweets
0,today ม็อบ18พฤศจิกา
1,โอ้ย น่ารักกกกก ปังมากแม่ นักข่าวมาเอาคลิปแบบน...
2,“ราษฎรโต้กลับ!”\n—\n18พฤศจิกาไปราษฎร์ประสงค์\n...
3,มันหมดยุคที่พวกมึงจะข่มขู่ไม่ให้คนไปม็อบด้วยกา...
4,Quote of the day ม็อบ18พฤศจิกา
5,แกนนำบอก ถ้าวันนี้ตำรวจใช้แก๊สน้ำตากับประชาชนอ...
6,รวมเจ้าเป็ดก้าบๆใน ม็อบ18พฤศจิกา (2)
7,แทนใจมาก “คนตจวอยากได้หอศิลป์” นอกจากงานเกษตรแ...
8,ชอบคำตอบพี่ทราย เจริญปุระเรื่องท่อน้ำเลี้ยงจัง...
9,ปืนฉีดน้ำเด็กอนุบาลหมีน้อย VS รถฉีดน้ำแรงดันสู...


In [19]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#create net col
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

In [20]:
df

,Tweets,Subjectivity,Polarity
0,สีสันของประชาธิปไตยและคนที่มีอุดมการณ์เดียวกัน...,0.0,0.00
1,เออจริงของพี่ทราย555555 \nแฉ ม็อบ18พฤศจิกา,0.0,0.00
2,นี่เป็นลมในม็อบ ถูกส่งขึ้นAmbulanceโผล่อีกทีที...,0.0,0.00
3,งานอาร์ต​ของพระองค์​ ทรงศิริโฉมมากเพคะ​ 🙏 ขอพร...,0.0,0.00
4,flghting \nflghting \nThe whole world is watch...,0.4,0.25
...,...,...,...
95,เห็นกี่ครั้งก็ขนลุกคนเยอะมาก ยิ่งพวกมันใช้วิธี...,0.0,0.00
96,ม็อบประเทศไทยนี่มันโคตรสุดเลย สุดเรื่องความกวน...,0.0,0.00
97,ไม่มีตำรวจ = ไม่มีความรุนแรง\nไม่มีรถฉีดน้ำ =...,0.0,0.00
98,คนไปม็อบไม่เเผ่วเลย ไปกันเยอะมาก ในเวลาไม่กี่ว...,0.0,0.00


In [21]:
df.to_csv(r'sentiment.csv', index = False)